In [13]:
using CorpusLoaders, MultiResolutionIterators, Base.Iterators
using Flux: onehotbatch, onecold
using Printf, Languages, DataStructures, Flux, BSON

In [3]:
dataset_train_pos = load(IMDB("train_pos"))
dataset_test_pos = load(IMDB("test_pos"))
dataset_train_neg = load(IMDB("train_neg"))
dataset_test_neg = load(IMDB("test_neg"))

Channel{Array{Array{String,1},1}}(sz_max:4,sz_curr:4)

In [4]:
docs_train_pos = collect(take(dataset_train_pos, 500));
docs_train_neg = collect(take(dataset_train_neg, 500));
docs_test_pos = collect(take(dataset_test_pos, 500));
docs_test_neg = collect(take(dataset_test_neg, 500));

In [5]:
docs_train_pos = flatten_levels(docs_train_pos, lvls(IMDB, :documents))|>full_consolidate;
docs_train_neg = flatten_levels(docs_train_neg, lvls(IMDB, :documents))|>full_consolidate;
docs_test_pos = flatten_levels(docs_test_pos, lvls(IMDB, :documents))|>full_consolidate;
docs_test_neg = flatten_levels(docs_test_neg, lvls(IMDB, :documents))|>full_consolidate;

In [70]:
list_stopwords = stopwords(Languages.English())
#push!(list_stopwords, "")

490-element Array{String,1}:
 "a"         
 "about"     
 "above"     
 "across"    
 "after"     
 "again"     
 "against"   
 "all"       
 "almost"    
 "alone"     
 "along"     
 "already"   
 "also"      
 ⋮           
 "young"     
 "younger"   
 "youngest"  
 "your"      
 "you're"    
 "yours"     
 "yourself"  
 "yourselves"
 "you've"    
 "z"         
 ""          
 ""          

In [48]:
function removeStopWords(tokens)
    filtered_sentence = []
    for token in tokens
        if  !(lowercase(token) in list_stopwords)
            push!(filtered_sentence, lowercase(token))
        end
    end
            
    return filtered_sentence
end

removeStopWords (generic function with 1 method)

In [66]:
using Unicode
function convert_clean_arr(arr)
    arr = string.(arr)
    arr = Unicode.normalize.(arr, stripmark=true)
    arr = map(x -> replace(x, r"[^a-zA-Z0-9_]" => ""), arr)
    return arr
end

convert_clean_arr (generic function with 1 method)

In [73]:
docs_train_pos = convert_clean_arr.(docs_train_pos);
docs_train_neg = convert_clean_arr.(docs_train_neg);
docs_test_pos = convert_clean_arr.(docs_test_pos);
docs_test_neg = convert_clean_arr.(docs_test_neg);

#--------------------------------------------------

docs_train_pos = removeStopWords.(docs_train_pos);
docs_train_neg = removeStopWords.(docs_train_neg);
docs_test_pos = removeStopWords.(docs_test_pos);
docs_test_neg = removeStopWords.(docs_test_neg);

In [74]:
train_label_pos = ones(Int64, size(docs_train_pos)[1]);
train_label_neg = zeros(Int64,size(docs_train_neg)[1]);
train_label = append!(train_label_pos, train_label_neg);

In [75]:
test_label_pos = ones(Int64, size(docs_test_pos)[1]);
test_label_neg = zeros(Int64,size(docs_test_neg)[1]);
test_label = append!(test_label_pos, test_label_neg);

In [41]:
length(test_label)

4112

In [76]:
train_set = [docs_train_pos; docs_train_neg];
test_set =  [docs_test_pos; docs_test_neg];

In [77]:
all_letters = collect(Iterators.flatten(train_set));
counter_letter = counter(all_letters);

In [78]:
vocab = Dict()
index = 1
for (item,v) in counter_letter
    vocab[lowercase(item)] = index
    index = index + 1
end

In [79]:
reviews_index_vocab = []
for review in train_set
    r = [get(vocab, lowercase(w), 0) for w in review]
    push!(reviews_index_vocab,r)
end

In [80]:
test_index_vocab = []
for review in test_set
    r = [get(vocab, lowercase(w), 0) for w in review]
    push!(test_index_vocab,r)
end

In [81]:
function pad_features(reviews_int, length_max)
    features = []
    for review_int in reviews_int
        dim_review = size(review_int)[1]
        pad_size = length_max-dim_review
        if pad_size > 0
            pad_array = zeros(Int64, pad_size)
            result = append!(pad_array,review_int)
        else
            result = review_int[1:length_max]
        end
        push!(features, result)
    end
    return features
end

pad_features (generic function with 1 method)

In [82]:
new_features = pad_features(reviews_index_vocab, 300);
new_test_features = pad_features(test_index_vocab, 300);

In [36]:
length(train_set) 

4158

In [88]:
model = Chain(
LSTM(300, 128),
LSTM(128,10),
Dense(10, 2),
softmax)

Chain(Recur(LSTMCell(300, 128)), Recur(LSTMCell(128, 10)), Dense(10, 2), NNlib.softmax)

In [89]:
# `loss()` calculates the crossentropy loss between our prediction `y_hat`
# (calculated from `model(x)`) and the ground truth `y`.  We augment the data
# a bit, adding gaussian random noise to our image to make it more robust.
function testeL(xs, ys)
    l = sum((model(xs)-ys).^2)
    return l
end

newloss(x, y) = Flux.mse(model(x), y)

accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

# Train our model with the given training set using the ADAM optimizer and
# printing out performance against the test set as we go.
opt = ADAM(0.001)

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

In [85]:
train_set_full = [ (new_features[i], train_label[i])  for i = 1:size(new_features)[1]];
test_set_full = [ (new_test_features[i], test_label[i])  for i = 1:size(new_test_features)[1]];

In [90]:
prediction(i) = findmax(model(new_test_features[i]))[2]-1

prediction (generic function with 1 method)

In [39]:
length(test_label)

6329

In [77]:
function create_batch(r, features, labels)
    xs = [ vec(Float64.(feature)) for feature in features]
    ys = [ label for label in labels[r]]
    return (Flux.batch(xs), Flux.batch(ys))
end

create_batch (generic function with 1 method)

In [78]:
trainbatch = create_batch(1:length(train_label), new_features, train_label);
testbatch = create_batch(1:length(test_label), new_test_features, test_label);

In [63]:
trainbatch[1]

500×4158 Array{Int64,2}:
     0      0      0      0      0  …      0      0      0      0      0
     0      0      0      0      0         0      0      0      0      0
     0      0      0      0      0         0      0      0      0      0
     0      0      0      0      0         0      0      0      0      0
     0      0      0      0      0         0      0      0      0      0
     0      0      0      0      0  …      0      0      0      0      0
     0      0      0      0      0         0      0      0      0      0
     0      0      0      0      0         0      0      0      0      0
     0      0      0      0      0         0      0      0      0      0
     0      0      0      0      0         0      0      0      0      0
     0      0      0      0      0  …      0      0      0      0      0
     0      0      0      0      0         0      0      0      0      0
     0      0      0      0      0         0      0      0      0      0
     ⋮                    

In [91]:

@info("Beginning training loop...")
best_acc = 0.0
last_improvement = 0
for epoch_idx in 1:200
    
    global best_acc, last_improvement
    # Train for a single epoch
    Flux.train!(newloss, params(model), train_set_full, opt)

    # Calculate accuracy:
    acc = sum(prediction(i) == test_label[i] for i in 1:length(test_label))/length(test_label)
    @info(@sprintf("[%d]: Test accuracy: %.4f", epoch_idx, acc))
    
    # If our accuracy is good enough, quit out.
    if acc >= 0.999
        @info(" -> Early-exiting: We reached our target accuracy of 99.9%")
        break
    end

    # If this is the best accuracy we've seen so far, save the model out
    if acc >= best_acc
        @info(" -> New best accuracy! Saving model out to mnist_conv.bson")
        BSON.@save "mnist_conv.bson" model epoch_idx acc
        best_acc = acc
        last_improvement = epoch_idx
    end

    # If we haven't seen improvement in 5 epochs, drop our learning rate:
    if epoch_idx - last_improvement >= 5 && opt.eta > 1e-6
        opt.eta /= 10.0
        @warn(" -> Haven't improved in a while, dropping learning rate to $(opt.eta)!")

        # After dropping learning rate, give it a few epochs to improve
        last_improvement = epoch_idx
    end

    if epoch_idx - last_improvement >= 10
        @warn(" -> We're calling this converged.")
        break
    end
end


┌ Info: Beginning training loop...
└ @ Main In[91]:2
┌ Info: [1]: Test accuracy: 0.4854
└ @ Main In[91]:13
┌ Info:  -> New best accuracy! Saving model out to mnist_conv.bson
└ @ Main In[91]:23
┌ Info: [2]: Test accuracy: 0.5013
└ @ Main In[91]:13
┌ Info:  -> New best accuracy! Saving model out to mnist_conv.bson
└ @ Main In[91]:23
┌ Info: [3]: Test accuracy: 0.4786
└ @ Main In[91]:13
┌ Info: [4]: Test accuracy: 0.4894
└ @ Main In[91]:13
┌ Info: [5]: Test accuracy: 0.4801
└ @ Main In[91]:13
┌ Info: [6]: Test accuracy: 0.4902
└ @ Main In[91]:13
┌ Info: [7]: Test accuracy: 0.4885
└ @ Main In[91]:13
┌ Warning:  -> Haven't improved in a while, dropping learning rate to 0.0001!
└ @ Main In[91]:32
┌ Info: [8]: Test accuracy: 0.4949
└ @ Main In[91]:13
┌ Info: [9]: Test accuracy: 0.4918
└ @ Main In[91]:13
┌ Info: [10]: Test accuracy: 0.4893
└ @ Main In[91]:13
┌ Info: [11]: Test accuracy: 0.4915
└ @ Main In[91]:13
┌ Info: [12]: Test accuracy: 0.4895
└ @ Main In[91]:13
┌ Warning:  -> Haven't impr

In [195]:
train_loss = Float64[]
test_loss = Float64[]
function update_loss!()
    push!(train_loss, newloss(train_set_full))
    push!(test_loss, newloss(test_set_full))
    @printf("train loss = %.2f, test loss = %.2f\n", train_loss[end], test_loss[end])
end

#Flux.train!(newloss, params(model), train_set_full, opt)

Flux.train!(newloss, params(model), Iterators.repeated(train_set_full, 1000), opt;
            cb = Flux.throttle(update_loss!, 1))
using Plots
plot(1:length(train_loss), train_loss, xlabel="~seconds of training", ylabel="loss", label="train")
plot!(1:length(test_loss), test_loss, label="test")

MethodError: MethodError: no method matching newloss(::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int64,1},Int64}, ::Tuple{Array{Int

In [109]:
newloss(train_set_full[1:10])

MethodError: MethodError: no method matching newloss(::Array{Tuple{Array{Int64,1},Int64},1})
Closest candidates are:
  newloss(::Any, !Matched::Any) at In[95]:9

In [131]:
count = 0
for i in test_set_full
    if i[2] == 0
        println("Neg ", count)
    end
    count = count +1
end

Neg 144
Neg 145
Neg 146
Neg 147
Neg 148
Neg 149
Neg 150
Neg 151
Neg 152
Neg 153
Neg 154
Neg 155
Neg 156
Neg 157
Neg 158
Neg 159
Neg 160
Neg 161
Neg 162
Neg 163
Neg 164
Neg 165
Neg 166
Neg 167
Neg 168
Neg 169
Neg 170
Neg 171
Neg 172
Neg 173
Neg 174
Neg 175
Neg 176
Neg 177
Neg 178
Neg 179
Neg 180
Neg 181
Neg 182
Neg 183
Neg 184
Neg 185
Neg 186
Neg 187
Neg 188
Neg 189
Neg 190
Neg 191
Neg 192
Neg 193
Neg 194
Neg 195
Neg 196
Neg 197
Neg 198
Neg 199
Neg 200
Neg 201
Neg 202
Neg 203
Neg 204
Neg 205
Neg 206
Neg 207
Neg 208
Neg 209
Neg 210
Neg 211
Neg 212
Neg 213
Neg 214
Neg 215
Neg 216
Neg 217
Neg 218
Neg 219
Neg 220
Neg 221
Neg 222
Neg 223
Neg 224
Neg 225
Neg 226
Neg 227
Neg 228
Neg 229
Neg 230
Neg 231
Neg 232
Neg 233
Neg 234
Neg 235
Neg 236
Neg 237
Neg 238
Neg 239
Neg 240
Neg 241
Neg 242
Neg 243
Neg 244
Neg 245
Neg 246
Neg 247
Neg 248
Neg 249
Neg 250
Neg 251
Neg 252
Neg 253
Neg 254
Neg 255
Neg 256
Neg 257
Neg 258
Neg 259
Neg 260
Neg 261
Neg 262
Neg 263
Neg 264
Neg 265
Neg 266
Neg 267
Neg 268


In [179]:
count = 1
for i in train_set_full
    result = model(train_set_full[300][1])
    if result == 1
        println("Pos " , count)
    end
    count = count +1
end

In [201]:
model(train_set_full[210][1])

2-element Array{Float32,1}:
 0.49534276
 0.50465727

In [189]:
using Flux : onecold
onecold(model(train_set_full[1][1]))
#mean(onecold(model()) .== onecold([1]))

UndefVarError: UndefVarError: onecold not defined

In [204]:
findmax(model(new_test_features[300]))[2]-1

0